<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 3: Future projections for each impact

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 4 included the
manuscript <<< insert doi >>>. Future risks for fire. We show median changes in
risk with respect to the historic.


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import rioxarray
from carbonplan.data import cat
import matplotlib.gridspec as gridspec

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
coarsen = 4

In [ ]:
mask = (
    (
        load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band")
        > 0.25
    )
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)

In [ ]:
region_bboxes = {
    "PNW": {"x": slice(-2.5e6, -1e6), "y": slice(3.5e6, 2.4e6)},
    "Southwest": {"x": slice(-1.8e6, -0.9e6), "y": slice(1.8e6, 0.9e6)},
    "California": {"x": slice(-2.3e6, -1.8e6), "y": slice(2.5e6, 1.2e6)},
    "Southeast": {"x": slice(0.6e6, 1.8e6), "y": slice(1.6e6, 0.3e6)},
}

# Load in fire data


In [ ]:
historical_fire = xr.open_zarr(
    get_store(
        "carbonplan-forests", "risks/results/paper/fire_terraclimate.zarr"
    )
).load()
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
ds = (
    xr.open_zarr(
        get_store("carbonplan-forests", "risks/results/paper/fire_cmip.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(fire_mask)
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
historical_annual = (
    xr.open_zarr(
        get_store(
            "carbonplan-forests",
            "risks/results/paper/fire_terraclimate.zarr",
        )
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
def load_regional_results(ds, timestep, region):
    selected = ds.sel(**region)
    if timestep == "annual":
        return (
            selected.mean(dim=["x", "y"])
            .rolling(dim={"year": 10})
            .mean()
            .compute()
        )
    elif timestep == "decadal":
        return selected.mean(dim=["x", "y"]).compute()

In [ ]:
results_dict = {"fire": {}, "insects": {}, "drought": {}}

In [ ]:
for region, bbox in region_bboxes.items():
    results_dict["fire"][region] = {}
    results_dict["fire"][region]["future"] = (
        load_regional_results(ds, "annual", region_bboxes[region])
        .sel(year=slice(1970, 2099))
        .probability
    )
    results_dict["fire"][region]["historical"] = load_regional_results(
        historical_annual, "annual", region_bboxes[region]
    ).historical

### Load in insect and drought data


In [ ]:
for impact in ["insects", "drought"]:
    historical_ds = (
        xr.open_zarr(
            get_store(
                "carbonplan-forests",
                "risks/results/paper/{}_terraclimate.zarr".format(impact),
            )
        )
        .assign_coords({"year": np.arange(1995, 2020, 10)})
        .compute()
    )
    future_ds = (
        xr.open_zarr(
            get_store(
                "carbonplan-forests",
                "risks/results/paper/{}_cmip.zarr".format(impact),
            )
        )
        .assign_coords({"year": np.arange(1975, 2100, 10)})
        .compute()
    )

    for region, bbox in region_bboxes.items():
        results_dict[impact][region] = {}
        results_dict[impact][region]["future"] = (
            load_regional_results(future_ds, "decadal", region_bboxes[region])
            .sel(year=slice(1970, 2099))
            .probability
        )
        results_dict[impact][region]["historical"] = load_regional_results(
            historical_ds, "decadal", region_bboxes[region]
        ).probability

## Plot the figure


In [ ]:
plot.multipanel_ts(results_dict, region_bboxes, "Figure-4")